In [1]:
#Logistic Regression Example, TensorFlow Integration with Julia Test
# from - https://github.com/malmaud/TensorFlow.jl
using TensorFlow


In [2]:
using Distributions

# Generating synthetic data

x = randn(100, 50)
w = randn(50, 10)
y_prob = exp(x * w)
y_prob ./= sum(y_prob, 2)

#drawing from sample
function draw(probs)
    y = zeros(size(probs))
    for i in 1:size(probs,1)
        index = rand(Categorical(probs[i, :]))
        y[i, index] = 1
    end
    return y
end

y = draw(y_prob)

100×10 Array{Float64,2}:
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮                      
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  1.0  0.0 

In [3]:
#Building the Model

sess = Session(Graph())
X = placeholder(Float64)
Y_obs = placeholder(Float64)

variable_scope("logistic_model", initializer = Normal(0, 0.001)) do
    global W = get_variable("weights", [50, 10], Float64)
    global B = get_variable("bias", [10], Float64)
end

Y = nn.softmax(X*W + B)
Loss = -reduce_sum(log(Y).*Y_obs)
optimizer = train.AdamOptimizer()
minimize_op = train.minimize(optimizer, Loss)
saver = train.Saver()



W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX2 instructions, but these are available on your machine and could speed up CPU computations.
W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use FMA instructions, but these are available on your machine and could speed up CPU computations.


<Saver>

In [4]:
#Running the training

run(sess, initialize_all_variables)
checkpoint_path = mktempdir()
info("Checkpoint files saved in $checkpoint_path")

for epoch in 1:100
    cur_loss, _ = run(sess, vcat(Loss, minimize_op), Dict(X=>x, Y_obs=>y))
    println(@sprintf("Current loss is %.2f", cur_loss))
    train.save(saver, sess, joinpath(checkpoint_path, "logistic"), global_step=epoch)
end

LoadError: MethodError: no method matching get_tensors(::TensorFlow.#initialize_all_variables)[0m
Closest candidates are:
  get_tensors([1m[31m::TensorFlow.nn.rnn_cell.LSTMStateTuple[0m) at /home/nikhil/.julia/v0.5/TensorFlow/src/ops/rnn_cell.jl:65
  get_tensors([1m[31m::TensorFlow.IndexedSlices[0m) at /home/nikhil/.julia/v0.5/TensorFlow/src/run.jl:41
  get_tensors([1m[31m::Array{T,1}[0m) at /home/nikhil/.julia/v0.5/TensorFlow/src/run.jl:26
  ...[0m